In [15]:
# 我们要定位文件位置，使用这里的代码，具体要将此notebook文件和function文件（或文件夹）放在同一个文件夹路径下，具体路径是什么可以左侧找到目标文件夹然后右击复制文件夹路径来获得
import sys
sys.path.insert(0,'/content/drive/MyDrive/JPMorgan/自写代码/TensorFlow_codes')

In [ ]:
pip install einops

In [ ]:
pip install wandb

In [ ]:
pip install pytorch_lightning

In [ ]:
pip install pykeops

In [20]:
pip install -q -U tensorflow-addons

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import os
import argparse
import json
import numpy as np
#------------------------------------------------------------------
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow_probability.python.internal import tensor_util
from tensorflow.keras.models import Sequential
#from keras.models import load_model
import torch
#import torch.nn as nn
#------------------------------------------------------------------
from utils.util import find_max_epoch, print_size, training_loss, calc_diffusion_hyperparams, std_normal
from utils.util import get_mask_mnr, get_mask_bm, get_mask_rm

from imputers.DiffWaveImputer import DiffWaveImputer
from imputers.SSSDSAImputer import SSSDSAImputer
from imputers.SSSDS4Imputer import SSSDS4Imputer

import easydict

In [ ]:
#Check that you have the last version of tensorflow
tf.__version__

'2.9.2'

In [ ]:
# We can check the train data's dimensions. It is a 3-D dataset
# Mujoco 数据中，第一维度(0): 8000代表8000个时间序列，或者8000次实验轨道。因为此数据集一共10,000个序列，根据80%作为train数据，20%作为test数据而分配到了8000次实验的train数据
# 第二维度(1): 100表示每次实验中的100个时间点(time points)
# 第三维度(2): 14表示14个features
# 上述分析均来自target paper原文中对Mujoco数据的解释
data = np.load("/content/drive/MyDrive/JPMorgan/自写代码/Data/Mujoco/train_mujoco.npy")
data.shape

(8000, 100, 14)

In [ ]:
data.shape[1:]

(100, 14)

In [22]:
class CustomFit(keras.Model):
  def __init__(self,model,only_generate_missing,loss_mask):
    super(CustomFit,self).__init__()
    self.model = model
    self.only_generate_missing = only_generate_missing
    self.loss_mask = loss_mask

  def compile(self, optimizer): #loss
    super(CustomFit, self).compile()
    self.optimizer = optimizer
    #self.loss = loss


  def train_step(self, data):
    x, y = data

    with tf.GradientTape() as tape:
      y_pred = self.model(x , training = True)
      if self.only_generate_missing == 1:
        loss_mask = torch.tensor(self.loss_mask.numpy(),dtype=torch.bool)
        y_pred = torch.tensor(y_pred.numpy())
        y_pred = tf.convert_to_tensor(y_pred[loss_mask].numpy().reshape((50,1260)),dtype=tf.float32)
        loss = keras.losses.mean_squared_error(y, y_pred)
      elif self.only_generate_missing == 0:
        #y_pred = tf.reshape(y_pred,[50,1260])
        loss = keras.losses.mean_squared_error(y, y_pred)

    #loss = loss
    training_vars = self.trainable_variables
    gradients = tape.gradient(loss, training_vars)
    #print("loss_function:++++++++++:",loss)
    #print("training_vars:+++++++++:", training_vars) # Add
    #print("gradients:+++++++++:", gradients) # Add

    self.optimizer.apply_gradients(zip(gradients, training_vars))
    acc_metric = tf.keras.metrics.MeanSquaredError(name="mse")
    acc_metric.update_state(y, y_pred)
    #self.compiled_metrics.update_state(y, y_pred)

    return {"loss":loss, "mse":acc_metric.result()}




In [23]:
def train(output_directory,
          ckpt_iter,
          n_iters,
          iters_per_ckpt,
          iters_per_logging,
          learning_rate,
          use_model,
          only_generate_missing,
          masking,
          missing_k):
    
    """
    Train Diffusion Models

    Parameters:
    output_directory (str):         save model checkpoints to this path
    ckpt_iter (int or 'max'):       the pretrained checkpoint to be loaded; 
                                    automatically selects the maximum iteration if 'max' is selected
    data_path (str):                path to dataset, numpy array.
    n_iters (int):                  number of iterations to train
    iters_per_ckpt (int):           number of iterations to save checkpoint, 
                                    default is 10k, for models with residual_channel=64 this number can be larger
    iters_per_logging (int):        number of iterations to save training log and compute validation loss, default is 100
    learning_rate (float):          learning rate

    use_model (int):                0:DiffWave. 1:SSSDSA. 2:SSSDS4.
    only_generate_missing (int):    0:all sample diffusion.  1:only apply diffusion to missing portions of the signal
    masking(str):                   'mnr': missing not at random, 'bm': blackout missing, 'rm': random missing
    missing_k (int):                k missing time steps for each feature across the sample length.
    """

    # generate experiment (local) path
    local_path = "T{}_beta0{}_betaT{}".format(diffusion_config["T"],
                                              diffusion_config["beta_0"],
                                              diffusion_config["beta_T"]) 

    # Get shared output_directory ready
    output_directory = os.path.join(output_directory, local_path)
    if not os.path.isdir(output_directory):
        os.makedirs(output_directory)
        os.chmod(output_directory, 0o775)
    print("output directory", output_directory, flush=True)

    # map diffusion hyperparameters to gpu
    global diffusion_hyperparams  # Add
    for key in diffusion_hyperparams:
        if key != "T":
            diffusion_hyperparams[key] = diffusion_hyperparams[key] #[Zihao]: 这里从util.py获得超参数后，对于所有的不是T的超参数(e.g., "Beta","Alpha"),都分配到gpu上，即记为cuda

    # predefine model
    if use_model == 0:
        net = DiffWaveImputer(**model_config)  # [Zihao]: 这里引入了imputers文件夹内的提前预定好的三个模型：[0] CSDI, [1] SSSDSA [2] sssds4， net:torch network
    elif use_model == 1:                  # 这里的**model_config表示在不同imputers下的提前预定好的模型参数，不同imputer的参数略有不同，可以看
        net = SSSDSAImputer(**model_config)   # 下面的一个代码块内定义了：global model_config，获得的参数来自各自imputer模型的json结构文件中
    elif use_model == 2:                  # 的 'wavenet_config' (对SSSDSA模型是"sashimi_config")，在下面的代码块内会指定imputer json结构文件的路径来获得参数
        net = SSSDS4Imputer(**model_config)   # 这里的net是nn.Model,是一种torch模型
    else:
        print('Model chosen not available.')
    print_size(net)

    # <***>define optimizer
    #optimizer = tf.keras.optimizers.Adam(net.parameters(), lr=learning_rate)                       #[Zihao]: torch used
                                                            # 这是 Adam算法，一种随机最优化算法，给出可迭代的参数与学习率，进行converge，这里给出的
                                                            # 参数就是net.parameters(), 这是特定imputer模型的初始化参数，net在前面被定义为了一个torch model(已转为cuda),这是pytorch框架下的model,model.parameters()就会返回此model所有参数list
                                                            # lr是Adam算法的学习率参数，我们规定为learning_rate,来自于
                                                            # 对应imputer的json文件中的"train_config"
    # load checkpoint
    if ckpt_iter == 'max':          # 我们在imputer的json文件中设置的就是max，所以checkpoint会首先从我们储存的output_directory中寻找之前存储的检查点，避免花费大量时间继续构建结构
        ckpt_iter = find_max_epoch(output_directory)    # find_max_epoch来自util.py文件里面的class
    if ckpt_iter >= 0:            # 如果找到了存储在output_directory中的检查点：
        try:
            #net2 = Sequential()  # 我们在这里设定模型类型是Squential()
            # load file
            model_path = os.path.join(output_directory, '{}.weight'.format(ckpt_iter))  # 这里需要包括.h5 file名字，比如'models/model.h5', models是一个文件夹，path结尾是h5文件
            # feed the model with all config and weights
            model = net
            model.load_weights(model_path, by_name=False)                                                                             
            # net2.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(lr=learning_rate), metrics=["mse"])  # 这里的loss=loss第二个loss是我们下面设定好的loss function给出的loss值

            print('Successfully loaded model at iteration {}'.format(ckpt_iter))           # 到这里才算是将checkpoint里面所有参数重新读取到新的模型中了
        except:
            ckpt_iter = -1
            print('No valid checkpoint model found, start training from initialization try.')   # 如果没有找到checkpoint，意味着要从头训练模型结构，这时可以看到手动将ckpt_iter设定为-1，为的是下面的train过程，不同的ckpt_iter会有不同的结果
    else:
        ckpt_iter = -1
        print('No valid checkpoint model found, start training from initialization.')

        
        
    
    ### Custom data loading and reshaping ###
        
        
    training_data = np.load(trainset_config['train_data_path'])  # 'train_data_path'        # trainset_config 在下面的初始化里面有，表示从imputer的json结构中获得train data的路径
    training_data = np.split(training_data, 160, 0)     # 这个将会把 training_data 分为160个相等分段(batch)，0表现axis=0，沿着横轴(时间)分割，training_data是3-D矩阵数据，第一维度是实验轨道次数：8000(0)，第二维度是每个轨道的100个样本时间点(1), 第三维度是14个features(2),所以这里表示分割为160份数据，features数目保持不变，时间保持不变，轨道分为160份，每份8000/160个轨道实验
    training_data = np.array(training_data)          # 转为array
    training_data = tensor_util.convert_nonref_to_tensor(training_data)          # tf.convert_to_tensor             # 转为TF tensor, TF默认使用GPU环境进行运算所以不需要额外声明
    print('Data loaded')                                             

    # training
    n_iter = ckpt_iter + 1                 # 这里的train.py的训练过程，如果在上面已经找到了之前训练过的checkpoint,则在上面已经赋给model了checkpoint内的神经网络层的参数，比如SSSDS4模型我们之前设定的"n_iters": 500 (在json文件中)，产生的checkpoint有600.pkl
                                  # 那么这里的ckpt_iter = 600, 则n_iter = 601>下面的n_iters+1 = 501 所以直接不进行while循环，直接跳出不再训练。
                                  # 如果在上面的checkpoint内没有发现.pkl记忆点文件，则上面会赋值ckpt_iter = -1, 此时n_iter = 0,则会进行下面的while循环进行训练，每次训练结束会给n_iter+1,当循环500次的时候，末尾再加1变成n_iter=501 = n_iters + 1(我们设定的),此时跳出循环完成训练
    while n_iter < n_iters + 1:
        for batch in training_data:           # 在上面我们将training_data分为了160组数据，那么应该就是160个batch，进行批量训练
            print("This is loop: ", n_iter,"...")
            #print('batch[0]: ', batch[0])

            if masking == 'rm':           # 这里就是看missing的数据类型是什么，可以是'rm': random missing, 'mnr': missing not at random, 'bm': blackout missing, 在SSSDS4里我们目前设定的是rm
                transposed_mask = get_mask_rm(batch[0], missing_k)  # get_mask_rm从util.py中import的,样本为batch[0]，这是随便选的第一个batch为例子，维度只有时间和feature两个, 因为在util.py里面我们只需要用到sample.shape，shape的大小数值而已
            elif masking == 'mnr':
                transposed_mask = get_mask_mnr(batch[0], missing_k)  # get_mask_mnr从util.py中import的,样本为batch[0]，这是随便选的第一个batch为例子，维度只有时间和feature两个, 因为在util.py里面我们只需要用到sample.shape，shape的大小数值而已
            elif masking == 'bm':
                transposed_mask = get_mask_bm(batch[0], missing_k)  # get_mask_bm从util.py中import的,样本为batch[0]，这是随便选的第一个batch为例子，维度只有时间和feature两个, 因为在util.py里面我们只需要用到sample.shape，shape的大小数值而已

            mask = tf.transpose(transposed_mask, perm=[1, 0])    # transposed_mask是一个tensor,这个tf.transpose()method是将此tensor的维度转为transposed_mask原始维度(0,1)中，1变到现在的第一号位，0变到现在的第二号位置: (1,0),其实就是行变列，列变行，现在行有14个表示features，列有100个表示每个轨道的时间点            
            mask = tf.expand_dims(mask, axis=0)    # 增加一个维度，原本mask是二维，现在在位置0增加一个维度，值为1，也就是增加了batch的第一维度
            mask = tf.repeat(mask, repeats=batch.shape[0],axis = 0)    # 表示将原始mask重复，构成新的tensor,结构设定为重复batch第一维度：轨道数，以此来补充满每个batch内的所有轨道数: 8000/160,每列每行中一个元素: 0 or 1，batch是一个三维数组，后两维都保持不变，于是这里说明我们直接将mask加了一个维度即batch.size()[0](第一维度)
            loss_mask = tensor_util.convert_nonref_to_tensor(~np.array(mask,dtype=bool),dtype=tf.int32)  # tf.convert_to_tensor  #将mask的值都转为bool值，0与1，之后使用~运算符表示：按位取反，本来是0的改为1，本来是1的改为0，之后叫做loss_mask
              
            batch = tf.transpose(batch, perm=[0, 2, 1])    # 这里表示把batch的维度的后两个维度替换位置，现在第一维度轨道数保持不变，第二维度变成features，第三维度变成每个轨道的时间点数，此时和mask的三个维度重合
            
            print("Check shapes: ",batch.shape, mask.shape, loss_mask.shape)
            assert batch.shape == mask.shape == loss_mask.shape  # assert是一个检查维护过程，这里检查batch, mask和loss_mask的size(三个维度大小)是否都相等

            # back-propagation
            X_train = batch, batch, mask, loss_mask  # 这里在输入要计算loss function的自变量X, X[0] = batch, X[1] = batch, X[2] = mask. X[3] = loss_mask
            
            #loss, Y_train = training_loss(net, tf.keras.losses.MeanSquaredError(), X_train, diffusion_hyperparams,         # 这里使用的training_loss类在util.py文件中，返回的是loss值，即nn.MSE值，diffusion_hyperparams是计算出来的，来自util.py的calc_diffusion_hyperparams,在nn.MSE(a,b)中，a是预测值, b是真实值
            #                     only_generate_missing=only_generate_missing)  # 注意最后的only_generate_missing，看上面的介绍，这个值如果是0则表示使用所有样本进行diffusion后获得MSE，如果是1则表示只使用missing部分进行diffusion获得的MSE
            #==============================================================
            model = net
            loss_fn = tf.keras.losses.MeanSquaredError()
            X = X_train
            diffusion_hyperparams = diffusion_hyperparams
            only_generate_missing=only_generate_missing
            #----------------------------------------------------
            _dh = diffusion_hyperparams     # 计算出的扩散模型超参数
            T, Alpha_bar = _dh["T"], _dh["Alpha_bar"]            
            audio = X[0]
            cond = X[1]
            mask = X[2]
            loss_mask = X[3]

            B, C, L = audio.shape  # B is batchsize, C=1, L is audio length
            diffusion_steps = tf.random.uniform(minval=0, maxval=T, shape=(B, 1, 1), dtype=tf.int32)  # randomly sample diffusion steps from 1~T (random integers)
            diffusion_steps = diffusion_steps.numpy()
            z = std_normal(audio.shape)
            if only_generate_missing == 1:
                z = audio * mask + z * (1 - mask)

            #print(Alpha_bar)
            
            transformed_X = tf.math.sqrt(Alpha_bar[diffusion_steps]) * audio + tf.math.sqrt(1 - Alpha_bar[diffusion_steps]) * z  # compute x_t from q(x_t|x_0)
            diffusion_steps = tensor_util.convert_nonref_to_tensor(diffusion_steps,dtype=tf.int32) # tf.convert_to_tensor
            print(transformed_X.shape)
            
            epsilon_theta = model((transformed_X, cond, mask, tf.reshape(diffusion_steps,[B, 1])))  # predict \epsilon according to \epsilon_\theta
            
            if only_generate_missing == 1:
              loss_mask = torch.tensor(loss_mask.numpy(),dtype=torch.bool)
              epsilon_theta = torch.tensor(epsilon_theta.numpy())
              epsilon_theta = tf.convert_to_tensor(epsilon_theta[loss_mask].numpy().reshape((50,1260)),dtype=tf.float32) # tf.convert_to_tensor #tensor_util.convert_nonref_to_tensor
              #print('This is epsilon_theta: ++++++++++++++++++',epsilon_theta)
              
            #elif only_generate_missing == 0:
            #  epsilon_theta = tf.convert_to_tensor(epsilon_theta.numpy(),dtype=tf.float32)


            
            #print('====+++++++====== epsilon_theta: ', epsilon_theta)
            #print(z.shape)  # Add
            
            #print(cond.shape)
            #print(mask.shape)
            #print(tf.reshape(diffusion_steps,[B, 1]).shape)
            #print(loss_mask.shape)
            #print(epsilon_theta.shape)
            #print(loss_mask)
            #print(len(z))
            #print([only_generate_missing])

            #z = torch.tensor(z.numpy())  # 这里转为torch为了下面的loss_fn(z[loss_mask],epsilon_theta[loss_mask])
            loss_mask = torch.tensor(loss_mask.numpy(),dtype=torch.bool)
            #epsilon_theta = torch.tensor(epsilon_theta.numpy())
            
            print(model)  # 这里的net还是SSSD4模型
            model.trainable = True
            #print("trainable_weights:----------------------------------------------------------------------------------------------------", net.get_weights())

            #print(lzh)
            #net2 = Sequential([net])
            #net2 = keras.Model(inputs=inputs, outputs=outputs)  #这里我们定义net为Sequential()模型，为了下面的compile

            #print('This is input epsilon_theta: ',tf.convert_to_tensor(epsilon_theta[loss_mask].numpy()))
            #print('This is input z: ',tf.convert_to_tensor(z[loss_mask].numpy().reshape((50,1260))))
            #loss_mask2 = X[3]
            x_train = (transformed_X, cond, mask, tf.reshape(diffusion_steps,[B, 1])) # loss_mask2, tf.constant([only_generate_missing]*transformed_X.shape[0])
            

            if only_generate_missing == 1: 
              #net.compile(loss=loss_fn, optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=["mse"], run_eagerly=True)  # 这里的loss=loss第二个loss是我们上面设定好的loss function给出的loss值
              #print(tf.convert_to_tensor(epsilon_theta[loss_mask].numpy()).shape)
              #print(len(np.stack((transformed_X, cond, mask, tf.reshape(diffusion_steps,[B, 1])),axis=0)))
              #print(len(np.stack(z[loss_mask].numpy(),axis=0)))
              #input = transformed_X, cond, mask, tf.reshape(diffusion_steps,[B, 1])
              #print('TYPE: ',type(input))net
              y_train = tf.convert_to_tensor(z[loss_mask].numpy().reshape((50,1260)))

              training = CustomFit(model=model, only_generate_missing=only_generate_missing, loss_mask=X[3])
              training.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)) # 
              training.fit(x_train,y_train,batch_size = 50, epochs = 1)
            elif only_generate_missing == 0:
              y_train = z
              
              training = CustomFit(model=model, only_generate_missing=only_generate_missing, loss_mask=X[3])
              training.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)) # 
              training.fit(x_train,y_train,batch_size = 50, epochs = 1)
              #net.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=["mse"])  # 这里的loss=loss第二个loss是我们上面设定好的loss function给出的loss值
              #history = net.fit(tf.convert_to_tensor(epsilon_theta.numpy()),tf.convert_to_tensor(z.numpy()))
            #==============================================================

            if n_iter % iters_per_logging == 0:   # 默认是100，即当迭代次数是整百次的时候，我们这里print出已经进行了多少次迭代和loss值是多少
                print("iteration: {} ".format(n_iter))

            # save checkpoint
            if n_iter > 0 and n_iter % iters_per_ckpt == 0:  # 在进行了整百次训练的时候：
                #checkpoint_name1 = '{}.config'.format(n_iter)  #这是在给checkpoint文件(.pkl)进行命名,由于.h5在TF框架下无法存储subclassed的客户定制模型，我们随便起一个名字叫.h5py来存储
                checkpoint_name_weight = '{}.weight'.format(n_iter)
                
                #json_str = model.to_json()
                #with open(os.path.join(output_directory, checkpoint_name1),'w') as text_file:
                #  text_file.write(json_str)
                model.save_weights(os.path.join(output_directory, checkpoint_name_weight))

                #model.save(os.path.join(output_directory, checkpoint_name),save_format='tf')  # 存储路径为json文件内定义的output_directory，命名为checkpoint的文件名
                print('model weight at iteration %s is saved===============================' % n_iter)  # print出checkpoint以及完成储存

            n_iter += 1   # 迭代次数+1，进行下一次的迭代

   

In [24]:
if __name__ == "__main__":                  # __name__ == "__main__" 表示目前run的就是train_self_written1_SSSDS4.ipynb文件本身，使用的模型(即train，上面block里定义的)都来自这个文件,这个代码块代表train()模型初始化时候的给定参数
    parser = argparse.ArgumentParser()
    parser.add_argument('-c', '--config', type=str, default='/content/drive/MyDrive/JPMorgan/自写代码/TensorFlow_codes/config/config_SSSDS4.json',  
                        help='JSON file for configuration')  # 这里只是表示读取文件的位置等，当run这个block的时候显示的比较好看

    args = parser.parse_args("")

    with open(args.config) as f:   # 读取json参数内容
        data = f.read()   # 读取json参数内容

    config = json.loads(data)  # 读取json参数内容， 将变量config赋值为json形式下的data
    print(config)   

    train_config = config["train_config"]  # training parameters  #训练模型的参数，来自对应imputer的json参数

    global trainset_config
    trainset_config = config["trainset_config"]  # to load trainset  # train和test数据的路径参数，来自对应imputer的json参数

    global diffusion_config
    diffusion_config = config["diffusion_config"]  # basic hyperparameters  # 这是diffusion模型的初始化超参数，来自对应imputer的json参数，之后会带入util.py文件的calc_diffusion_hyperparams去计算出扩散模型的超参数

    global diffusion_hyperparams
    diffusion_hyperparams = calc_diffusion_hyperparams(
        **diffusion_config)  # dictionary of all diffusion hyperparameters  # 这里就是使用了util.py文件内的calc_diffusion_hyperparams,引入所有diffusion_config内的参数去计算出了扩散模型的超参数

    global model_config
    if train_config['use_model'] == 0:         # 这些是imputer对应模型的初始化超参数，分别对应0:DiffWave. 1:SSSDSA. 2:SSSDS4.
        model_config = config['wavenet_config']   # 给model_config赋值这些参数，之后可以带入模型进行imputer初始化，方便后续使用Adam算法进行迭代，见上面
    elif train_config['use_model'] == 1:
        model_config = config['sashimi_config']
    elif train_config['use_model'] == 2:
        model_config = config['wavenet_config']


{'diffusion_config': {'T': 200, 'beta_0': 0.0001, 'beta_T': 0.02}, 'wavenet_config': {'in_channels': 14, 'out_channels': 14, 'num_res_layers': 36, 'res_channels': 256, 'skip_channels': 256, 'diffusion_step_embed_dim_in': 128, 'diffusion_step_embed_dim_mid': 512, 'diffusion_step_embed_dim_out': 512, 's4_lmax': 100, 's4_d_state': 64, 's4_dropout': 0.0, 's4_bidirectional': 1, 's4_layernorm': 1}, 'train_config': {'output_directory': '/content/drive/MyDrive/JPMorgan/自写代码/TensorFlow_codes/Results_SSSDS4/Mujoco/train_90/', 'ckpt_iter': 'max', 'iters_per_ckpt': 100, 'iters_per_logging': 100, 'n_iters': 300, 'learning_rate': 0.0002, 'only_generate_missing': 0, 'use_model': 2, 'masking': 'rm', 'missing_k': 90}, 'trainset_config': {'train_data_path': '/content/drive/MyDrive/JPMorgan/自写代码/Data/Mujoco/train_mujoco.npy', 'test_data_path': '/content/drive/MyDrive/JPMorgan/自写代码/Data/Mujoco/test_mujoco.npy', 'segment_length': 100, 'sampling_rate': 100}, 'gen_config': {'output_directory': '/content/driv

In [25]:
# The start of the training.
# All the parameters are the already set ones in **train_config, it is from the json file we set, 
# here is "/content/drive/MyDrive/JPMorgan/自写代码/TensorFlow_codes/config/config_SSSDS4.json",
# the "**" means the "train" model will accept all the parameters in the json file above. So by this way,
# all the already defined parameters in the json file will by inputted into the "train" model 

# It will take lots of time when you first run it as it will generate the iteration saving checkpoint. So next
# time when you run it, it will just start from the last iteration memory checkpoint. The path you can find is
# in the json file called: "output_directory"

train(**train_config)

output directory /content/drive/MyDrive/JPMorgan/自写代码/TensorFlow_codes/Results_SSSDS4/Mujoco/train_90/T200_beta00.0001_betaT0.02
Successfully loaded model at iteration 300
Data loaded


In [ ]:
net = SSSDS4Imputer(**model_config)
net.load_weights('/content/drive/MyDrive/JPMorgan/自写代码/TensorFlow_codes/Results_SSSDS4/Mujoco/train_90/T200_beta00.0001_betaT0.02/1.weight',by_name=False)

In [ ]:
# Test whether GPU is using
!nvidia-smi

Fri Nov 18 17:23:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.cuda.is_available()


True

————————————————————————————————————————

The following is done by myself used for some exercises

————————————————————————————————————————

In [ ]:
a = np.arange(63000).reshape((50, 1260))
b = tf.convert_to_tensor(a)
b

<tf.Tensor: shape=(50, 1260), dtype=int64, numpy=
array([[    0,     1,     2, ...,  1257,  1258,  1259],
       [ 1260,  1261,  1262, ...,  2517,  2518,  2519],
       [ 2520,  2521,  2522, ...,  3777,  3778,  3779],
       ...,
       [59220, 59221, 59222, ..., 60477, 60478, 60479],
       [60480, 60481, 60482, ..., 61737, 61738, 61739],
       [61740, 61741, 61742, ..., 62997, 62998, 62999]])>

In [ ]:
c = np.arange(63000).reshape((50, 1260))
d = tf.convert_to_tensor(c)
mse = tf.keras.losses.MeanSquaredError()
mse(b,d)

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [ ]:
import tensorflow as tf
tf.constant([1]*10)[0]==1

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
aaa = tf.ones([100,14])
bbb = tf.expand_dims(aaa, axis=0)

In [ ]:
aaa.shape[0]

100

In [ ]:
aaa.numpy()

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [ ]:
tf.keras.losses.mean_squared_error([5],[8])

<tf.Tensor: shape=(), dtype=int32, numpy=9>

In [ ]:
mask2=~np.array(mask,dtype=bool)
mask2

array([ True, False, False,  True, False,  True])

In [ ]:
tf.convert_to_tensor(mask2)

<tf.Tensor: shape=(6,), dtype=bool, numpy=array([ True, False, False,  True, False,  True])>

In [ ]:
import torch
mask2 = torch.ones([100,14])
mask2.size(0)

100

In [ ]:
def std_normal(size):
    """
    Generate the standard Gaussian variable of a certain size
    """

    return torch.normal(0, 1, size=size).cuda()
z = std_normal((3,2))
z

tensor([[ 0.7820,  0.1527],
        [-0.4065,  0.0887],
        [-0.6603, -1.4359]], device='cuda:0')

In [ ]:
index = [[0,2],[0,0],[1,1]]
aaa = torch.tensor(index)
z[aaa]

tensor([[[ 0.7820,  0.1527],
         [-0.6603, -1.4359]],

        [[ 0.7820,  0.1527],
         [ 0.7820,  0.1527]],

        [[-0.4065,  0.0887],
         [-0.4065,  0.0887]]], device='cuda:0')

In [ ]:
aaa.shape

torch.Size([3, 2])

In [ ]:
import tensorflow as tf
def std_normal2(size):
    """
    Generate the standard Gaussian variable of a certain size
    """

    return tf.random.normal(mean=0, stddev=1, shape=size)
z2 = std_normal2((3,2))
z2

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[ 0.519981  , -0.28209618],
       [ 0.01872117, -1.0729865 ],
       [ 0.32988086,  0.2622309 ]], dtype=float32)>

In [ ]:
bbb = tf.convert_to_tensor(index)
for i in range(len(z2)):
  print(z2[i][bbb[i]])

TypeError: ignored

In [ ]:
z2[2]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.32988086, 0.2622309 ], dtype=float32)>

In [ ]:
bbb[0][0]

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [ ]:
mask3.size()

torch.Size([14, 100])

In [ ]:
aaa=tf.random.shuffle(range(5))[0:3].numpy()
aaa

array([4, 2, 0], dtype=int32)

In [ ]:
torch.tensor(range(10))

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
import torch
torch.randperm(len(torch.tensor(range(10))))

tensor([4, 1, 2, 5, 6, 9, 7, 8, 3, 0])

In [ ]:
print('aaa',aaa.shape)
print('bbb',bbb.shape)
print('ccc',ccc.shape)

aaa (2, 3)
bbb (1, 2, 3)
ccc (4, 2, 3)


In [ ]:
 import torch
 x = torch.tensor([[1, 2, 3],[4,5,6]])
 y = x.repeat(4, 1,1)
 y

tensor([[[1, 2, 3],
         [4, 5, 6]],

        [[1, 2, 3],
         [4, 5, 6]],

        [[1, 2, 3],
         [4, 5, 6]],

        [[1, 2, 3],
         [4, 5, 6]]])

In [ ]:
import tensorflow as tf
aaa=tf.convert_to_tensor([[1,2,3],[2,3,5]])
#aaa.numpy()

In [ ]:
aaa.shape[-1]

3

In [ ]:
print('x',x.shape)
print('y',y.shape)

x torch.Size([2, 3])
y torch.Size([4, 2, 3])


In [ ]:
aaa.repeat(3,5)

tensor([[2, 3, 5, 2, 3, 5, 2, 3, 5, 2, 3, 5, 2, 3, 5],
        [2, 3, 5, 2, 3, 5, 2, 3, 5, 2, 3, 5, 2, 3, 5],
        [2, 3, 5, 2, 3, 5, 2, 3, 5, 2, 3, 5, 2, 3, 5]])

In [ ]:
aaa.repeat(3, 1, 2)

tensor([[[2, 3, 5, 2, 3, 5]],

        [[2, 3, 5, 2, 3, 5]],

        [[2, 3, 5, 2, 3, 5]]])

In [ ]:
aaa.repeat(3, 1, 2).size()

torch.Size([3, 1, 6])

In [ ]:
aaa.repeat(3,5).shape[0]

3

In [ ]:
data = np.load("/content/drive/MyDrive/JPMorgan/自写代码/Data/Mujoco/train_mujoco.npy")

In [ ]:
data

In [ ]:

import torch
x = torch.randn(2, 3)
x

tensor([[-0.9415,  0.7483,  0.6444],
        [-0.1961,  0.2103, -1.0184]])

In [ ]:
x.view([3,2,1])

tensor([[[-0.9415],
         [ 0.7483]],

        [[ 0.6444],
         [-0.1961]],

        [[ 0.2103],
         [-1.0184]]])

In [ ]:
x.reshape([3,2,1])

tensor([[[-0.9415],
         [ 0.7483]],

        [[ 0.6444],
         [-0.1961]],

        [[ 0.2103],
         [-1.0184]]])

In [ ]:
tf.concat((y,y,y),1)

<tf.Tensor: shape=(2, 9), dtype=float32, numpy=
array([[-0.5305082 , -1.3207436 , -0.753209  , -0.5305082 , -1.3207436 ,
        -0.753209  , -0.5305082 , -1.3207436 , -0.753209  ],
       [ 0.47132334, -0.8071756 ,  0.13752413,  0.47132334, -0.8071756 ,
         0.13752413,  0.47132334, -0.8071756 ,  0.13752413]],
      dtype=float32)>

In [ ]:
import tensorflow as tf
aaa=tf.convert_to_tensor([[1,2,3],[2,3,5]])
bbb=tf.reshape(aaa,[3,2,1])
bbb

<tf.Tensor: shape=(3, 2, 1), dtype=int32, numpy=
array([[[1],
        [2]],

       [[3],
        [2]],

       [[3],
        [5]]], dtype=int32)>